In [64]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
import pandas as pd
import os
import xgboost as xgb

In [65]:
#load all dfs form a folder and merge them on ID
""" path = "/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/"
for i, file in enumerate(os.listdir(path)):
    if i == 0:
        df = pd.read_csv(path + file)
    else:
        df = pd.merge(df, pd.read_csv(path + file), on='ID') """
df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/aseg.volume_aparc.volume_aparc.thickness.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages.csv")

label_df = label_df[['ID', 'label_age_group']]

In [66]:
for col in df.columns:
    print(col)

ID
Left-Lateral-Ventricle
Left-Inf-Lat-Vent
Left-Cerebellum-White-Matter
Left-Cerebellum-Cortex
Left-Thalamus
Left-Caudate
Left-Putamen
Left-Pallidum
3rd-Ventricle
4th-Ventricle
Brain-Stem
Left-Hippocampus
Left-Amygdala
CSF
Left-Accumbens-area
Left-VentralDC
Left-vessel
Left-choroid-plexus
Right-Lateral-Ventricle
Right-Inf-Lat-Vent
Right-Cerebellum-White-Matter
Right-Cerebellum-Cortex
Right-Thalamus
Right-Caudate
Right-Putamen
Right-Pallidum
Right-Hippocampus
Right-Amygdala
Right-Accumbens-area
Right-VentralDC
Right-vessel
Right-choroid-plexus
5th-Ventricle
WM-hypointensities
Left-WM-hypointensities
Right-WM-hypointensities
non-WM-hypointensities
Left-non-WM-hypointensities
Right-non-WM-hypointensities
Optic-Chiasm
CC_Posterior
CC_Mid_Posterior
CC_Central
CC_Mid_Anterior
CC_Anterior
BrainSegVol
lhCortexVol
rhCortexVol
CortexVol
lhCerebralWhiteMatterVol
rhCerebralWhiteMatterVol
CerebralWhiteMatterVol
SubCortGrayVol
TotalGrayVol
SupraTentorialVol
SupraTentorialVolNotVent
lh_bankssts_volu

In [67]:
for col in label_df.columns:
    print(col)

ID
label_age_group


In [68]:
#merge the dataframes on ID
merged_df = pd.merge(df, label_df, on='ID', how='inner')

In [69]:
merged_df["label_age_group"].value_counts()
merged_df.dropna(inplace=True)
merged_df = merged_df[:5000]

#drop specific labels for label_age_group
merged_df = merged_df[merged_df.label_age_group != 10]

In [70]:
y = merged_df["label_age_group"]
X = merged_df.drop(["ID", "label_age_group"], axis=1)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [72]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, classification_report


# Initialize the classifier
clf = TabPFNClassifier()

# Fit the model
clf.fit(X_train, y_train)

# Predict probabilities
y_pred_proba = clf.predict_proba(X_test)

y_pred = clf.predict(X_test)
    






/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.v

In [85]:
y_pred_proba

array([[6.8006426e-02, 2.3731786e-01, 5.2714503e-01, 1.5932712e-01,
        8.1425626e-03, 6.0971695e-05],
       [1.9714711e-02, 1.4076862e-01, 6.0184282e-01, 2.2850320e-01,
        9.1365017e-03, 3.4197707e-05],
       [1.9087957e-02, 9.6702002e-02, 4.9293342e-01, 3.0362391e-01,
        8.7134972e-02, 5.1772047e-04],
       ...,
       [7.4865562e-01, 2.3717251e-01, 1.4094172e-02, 6.4441672e-05,
        1.9843835e-06, 1.1266379e-05],
       [1.2798468e-05, 3.3025342e-04, 4.2571291e-02, 4.2813656e-01,
        5.1963097e-01, 9.3181496e-03],
       [1.4870409e-04, 6.1067315e-03, 2.4943218e-01, 6.2254393e-01,
        1.2149446e-01, 2.7403640e-04]], dtype=float32)

In [87]:
y_pred_proba_control

array([[4.4328604e-02, 1.1596132e-01, 4.3027234e-01, 3.4184331e-01,
        6.5969616e-02, 1.6248203e-03],
       [7.9834503e-01, 1.8716162e-01, 1.4283202e-02, 1.7531322e-04,
        1.0635091e-05, 2.4257262e-05],
       [5.8953266e-04, 1.7152051e-02, 3.9571822e-01, 5.3615361e-01,
        5.0278697e-02, 1.0778865e-04],
       ...,
       [1.2077820e-02, 7.4445002e-02, 4.5496869e-01, 3.6415887e-01,
        9.3537904e-02, 8.1164378e-04],
       [7.6668406e-01, 2.1025951e-01, 2.2039991e-02, 8.5297041e-04,
        8.4699328e-05, 7.8813006e-05],
       [5.1594037e-05, 1.1093366e-03, 4.5414258e-02, 2.9041108e-01,
        6.4175522e-01, 2.1258553e-02]], dtype=float32)

In [76]:
y_test.shape

(1650,)

In [74]:
# ROC AUC for multiclass using 'macro' average
# ROC AUC for multiclass using 'macro' average
auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_test))
random_y_test = np.random.randint(0, n_classes, size=y_test.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred)

/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [75]:
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)

ROC AUC: 0.8672360971768458
Accuracy: 0.5363636363636364
Balanced Accuracy: 0.4515987180634366
Random Balanced Accuracy: 0.16962069218414957
              precision    recall  f1-score   support

         0.0       0.63      0.71      0.67       175
         1.0       0.46      0.30      0.37       207
         2.0       0.52      0.61      0.56       488
         3.0       0.47      0.47      0.47       450
         4.0       0.64      0.61      0.63       308
         5.0       0.00      0.00      0.00        22

    accuracy                           0.54      1650
   macro avg       0.45      0.45      0.45      1650
weighted avg       0.53      0.54      0.53      1650



In [11]:
#use xgboost
xg_clf = xgb.XGBClassifier()
xg_clf.fit(X_train, y_train)
y_pred = xg_clf.predict(X_test)
y_pred_proba = xg_clf.predict_proba(X_test)

In [12]:
# ROC AUC for multiclass using 'macro' average
auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_test))
random_y_test = np.random.randint(0, n_classes, size=y_test.shape)
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)

ROC AUC: 0.8309836256750626
Accuracy: 0.49454545454545457
Balanced Accuracy: 0.39428666244607385
Random Balanced Accuracy: 0.16409731610896594
              precision    recall  f1-score   support

         0.0       0.60      0.53      0.56       346
         1.0       0.37      0.19      0.25       405
         2.0       0.48      0.61      0.54       966
         3.0       0.45      0.47      0.46       903
         4.0       0.59      0.57      0.58       631
         5.0       0.00      0.00      0.00        49

    accuracy                           0.49      3300
   macro avg       0.41      0.39      0.40      3300
weighted avg       0.48      0.49      0.48      3300



/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [94]:
#verify on onther dataset
df_control = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/deconfounded_but_age/aseg.volume_aparc.thickness_aparc.volume.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/age_label/all_ages.csv")

label_df = label_df[['ID', 'label_age_group']]
df_control = df_control[df.columns]

In [95]:
#get the label and the data
merged_df_control = pd.merge(df_control, label_df, on='ID', how='inner')
merged_df_control.dropna(inplace=True)

X = merged_df_control.drop(["ID", "label_age_group"], axis=1)
y = merged_df_control["label_age_group"]

In [97]:
#traintest split but just getting train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=None, random_state=42, stratify=y)

In [101]:
len(X_train)

162

In [100]:
len(y_train)

162

In [102]:
#check that the colum order is the same for df and control give back false if not so
print(all(X.columns == X_train.columns))

True


In [103]:
y_pred_control = clf.predict(X_train)
y_pred_proba_control = clf.predict_proba(X_train)

/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/zi/home/esra.lenz/.conda/envs/NAKO_CLIP/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: Futur

In [104]:
print("y_score shape:", len(y_train))
print("y_true shape:", y_pred_proba_control.shape)
print("y_pred shape:", y_pred_control.shape)

y_score shape: 162
y_true shape: (162, 6)
y_pred shape: (162,)


In [105]:
#ROC
#auc = roc_auc_score(y_control, y_pred_proba_control, multi_class='ovr', average='macro')
auc_control = roc_auc_score(y_train, y_pred_proba_control, multi_class='ovr', average='macro')
acc = accuracy_score(y_train, y_pred_control)
balanced_acc = balanced_accuracy_score(y_train, y_pred_control)
report = classification_report(y_train, y_pred_control)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_train))
random_y_test = np.random.randint(0, n_classes, size=y_train.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred_control)

#print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [88]:
# ROC AUC for multiclass using 'macro' average
#auc = roc_auc_score(y_control, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_control, y_pred)
balanced_acc = balanced_accuracy_score(y_control, y_pred)
report = classification_report(y_control, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_control))
random_y_test = np.random.randint(0, n_classes, size=y_control.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred)

ValueError: Found input variables with inconsistent numbers of samples: [217, 1650]

In [ ]:
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)